In [ ]:
qb = QuantBook()
dt = (2022, 10, 12) # This is the end date for the data you collect
qb.SetStartDate(*dt)
# symbol = qb.AddIndex("VIX", Resolution.Minute).Symbol # Use line below for Equities
symbol = qb.AddEquity("TQQQ", Resolution.Minute).Symbol
price = qb.History([symbol], datetime(*dt) - timedelta(1), qb.StartDate, Resolution.Minute)
price = price.open.values[0] # Only important if you want to restrict strikes to +- the current price
# weekly_canonical_symbol = Symbol.CreateCanonicalOption(symbol, "VIXW", Market.USA, "?VIXW") # This line is to get weekly options for indexes
# contracts = qb.OptionChainProvider.GetOptionContractList(weekly_canonical_symbol, datetime(*dt) - timedelta(1)) # This line can also be hashed out for equities, use line below
contracts = qb.OptionChainProvider.GetOptionContractList(symbol, datetime(*dt))
# exps = sorted(contracts, key=lambda x : x.ID.Date) - Not important line, leave hashed out unless debugging
expDates = set([exps[i].ID.Date for i in range(len(exps))])
expiry = exps[0].ID.Date
contracts = [contract for contract in contracts if contract.ID.Date == expiry] 
    # and contract.ID.StrikePrice > int(0.85 * price) and contract.ID.StrikePrice < int(1.15 * price)] - This line is for restricting the number of strikes
for contract in contracts:
    option = qb.AddOptionContract(contract, Resolution.Minute)
    option.PriceModel = OptionPriceModels.BlackScholes() # Change this for American options - Could use OptionPriceModels.CrankNicolsonFD() to start

In [ ]:
pd.set_option('display.max_colwidth', None)
# Reset the volatility model
qb.Securities[symbol].VolatilityModel = StandardDeviationOfReturnsVolatilityModel(30, Resolution.Minute)

# Get historical data
requests = []
for security in sorted(qb.Securities.Values, key=lambda x: x.Type):
    for subscription in security.Subscriptions:
        requests.append(HistoryRequest(subscription, security.Exchange.Hours, qb.StartDate-timedelta(1), qb.StartDate)) # timedelta sets history from dt - 1 day to now
history = qb.History(requests)

df = pd.DataFrame()

# Iterate through the historical data
HasVals = False
HeaderPrinted = False
i = 0
for slice in history:
    underlying_price = None
    underlying_volatility = None

    for bar in slice.QuoteBars.Values:
        qb.Securities[bar.Symbol].SetMarketPrice(bar)

    for bar in slice.Bars.Values:
        symbol = bar.Symbol
        security = qb.Securities[symbol]
        security.SetMarketPrice(bar)
        # Comment this if statement for Equities (Market Price will show up in loop as index and this will lead to no outputs if I remember right)
        """ if security.Type == SecurityType.Index: 
            underlying_volatility = security.VolatilityModel.Volatility
            underlying_price = security.get_Price()
            continue """
        # Uncomment all this for Equities
        if security.Type == SecurityType.Equity:
            underlying_volatility = security.VolatilityModel.Volatility
            underlying_price = security.Price
            continue 
        contract = OptionContract.Create(symbol, symbol.Underlying, bar.EndTime, security, underlying_price) # Uncomment this for Equities
        # contract = OptionContract.Create(symbol, security.get_Symbol(), bar.EndTime, security, underlying_price) # Comment This for Equities
        contract.LastPrice = security.Close # Alternatively use bar.Close for Last Trade, this uses bid/ask midpoint
        lastPrice = contract.LastPrice
        result = security.PriceModel.Evaluate(security, None, contract)
        greeks = result.Greeks
        
        # Append to dataframe
        data = {
            "IV" : result.ImpliedVolatility,
            "Delta": greeks.Delta,
            "Gamma": greeks.Gamma,
            "Bid": contract.BidPrice,
            "Bid Size": contract.BidSize,
            "Ask": contract.AskPrice,
            "Ask Size": contract.AskSize,
            "underlyingPrice": underlying_price
        }
        right = "Put" if symbol.ID.OptionRight == 1 else "Call"
        index = pd.MultiIndex.from_tuples([(symbol.ID.Date, symbol.ID.StrikePrice, right, bar.EndTime)], names=["expiry", "strike", "type", "endTime"])
        df = pd.concat([df, pd.DataFrame(data, index=index)])
    if(HasVals == False):
        dfcsv = df.to_csv()
        if(dfcsv != '""\n'):
            HasVals = True
    if(HasVals):
        i += 1
        if(i % 3 == 0):
            if not HeaderPrinted:
                string = df.to_csv(header=True)
                HeaderPrinted = True
            else:
                string = df.to_csv(header=False)
            string = string.replace("\n", "<br>")
            print(string, end = "<br>")
            df = df.iloc[0:0]
            if(i >= int(6.5 * 60)): # i here reps minutes from start. 
                print("Set break here") # Set your breakpoint here, then copy text output to Notepad++